In [1]:
import pandas as pd
import os

In [11]:
prefix_list=['op_math_', 'op_math_sorted_', 'op_gsm8k_sorted', 'op_gsm8k_math_sorted', 'op_gsm8k_math','op_combined_']
prefix_list = sorted(prefix_list, key=len, reverse=True)
dic={p:[] for p in prefix_list}

In [3]:
dic

{'op_gsm8k_math_sorted': [],
 'op_math_sorted_': [],
 'op_gsm8k_sorted': [],
 'op_gsm8k_math': [],
 'op_combined_': [],
 'op_math_': []}

In [4]:
cd drive/MyDrive/685_models

/content/drive/MyDrive/685_models


In [12]:
for file in os.listdir('inf_op_re'):
    for prefix in prefix_list:
      if file.startswith(prefix) and file.endswith(".json"):
        dic[prefix].append(os.path.join('inf_op_re',file))
        break




In [13]:
dic

{'op_gsm8k_math_sorted': ['inf_op_re/op_gsm8k_math_sorted_1_final.json',
  'inf_op_re/op_gsm8k_math_sorted_2_final.json',
  'inf_op_re/op_gsm8k_math_sorted_3_final.json'],
 'op_math_sorted_': ['inf_op_re/op_math_sorted_2_final.json',
  'inf_op_re/op_math_sorted_0_final.json',
  'inf_op_re/op_math_sorted_1_final.json'],
 'op_gsm8k_sorted': ['inf_op_re/op_gsm8k_sorted_3_final.json',
  'inf_op_re/op_gsm8k_sorted_1_final.json'],
 'op_gsm8k_math': ['inf_op_re/op_gsm8k_math_0_final.json',
  'inf_op_re/op_gsm8k_math_1_final.json',
  'inf_op_re/op_gsm8k_math_2_final.json'],
 'op_combined_': ['inf_op_re/op_combined_3.json',
  'inf_op_re/op_combined_5.json',
  'inf_op_re/op_combined_4.json'],
 'op_math_': ['inf_op_re/op_math_1_final.json',
  'inf_op_re/op_math_0_final.json',
  'inf_op_re/op_math_2_final.json']}

In [15]:
import json
with open('inf_op_re/op_math_2_final.json', 'r') as f:
            data = json.load(f)

In [36]:
import json
from collections import defaultdict
file_groups=dic

averages_by_prefix = {}

for prefix, files in file_groups.items():
    sum_accuracy = 0
    total_files = 0
    level_sums = defaultdict(float)
    level_counts = defaultdict(int)

    for file in files:
        with open(file, 'r') as f:
            data = json.load(f)


        sum_accuracy += data.get('accuracy', 0)
        total_files += 1


        for level, acc in data.get('accuracy_by_level', {}).items():
            level_sums[level] += acc
            level_counts[level] += 1


    avg_accuracy = sum_accuracy / total_files if total_files else 0


    avg_by_level = {
        level: level_sums[level] / level_counts[level]
        for level in level_sums
    }

    averages_by_prefix[prefix] = {
        'avg_accuracy': avg_accuracy,
        'avg_accuracy_by_level': avg_by_level
    }


for prefix, stats in averages_by_prefix.items():
    print(f"{prefix}:")
    print(f"  Overall accuracy: {stats['avg_accuracy']:.3f}")
    print(f"  Accuracy by level: {stats['avg_accuracy_by_level']}")


op_gsm8k_math_sorted:
  Overall accuracy: 0.138
  Accuracy by level: {'1': 0.3178294573643411, '2': 0.19259259259259257, '3': 0.1714285714285714, '4': 0.125, '5': 0.029850746268656716}
op_math_sorted_:
  Overall accuracy: 0.154
  Accuracy by level: {'1': 0.248062015503876, '2': 0.22592592592592595, '3': 0.22539682539682537, '4': 0.10677083333333333, '5': 0.06467661691542288}
op_gsm8k_sorted:
  Overall accuracy: 0.113
  Accuracy by level: {'1': 0.3023255813953488, '2': 0.2, '3': 0.10952380952380952, '4': 0.07421875, '5': 0.033582089552238806}
op_gsm8k_math:
  Overall accuracy: 0.177
  Accuracy by level: {'1': 0.3875968992248062, '2': 0.28888888888888886, '3': 0.20952380952380953, '4': 0.11458333333333333, '5': 0.06965174129353234}
op_combined_:
  Overall accuracy: 0.190
  Accuracy by level: {'1': 0.44961240310077516, '2': 0.3222222222222222, '3': 0.2634920634920635, '4': 0.11458333333333333, '5': 0.03233830845771144}
op_math_:
  Overall accuracy: 0.171
  Accuracy by level: {'1': 0.44961

In [19]:
import pickle
with open('stats.pkl', 'wb') as f:
    pickle.dump(averages_by_prefix, f)

In [35]:
df['accuracy']

KeyError: 'accuracy'

In [40]:
for p in dic.keys():
  if p=='op_gsm8k_sorted':
    continue
  dicts=[]
  for file in dic[p]:
    df= pd.read_csv(file)
    accuracy_by_level = df.groupby('level')['is_correct'].agg(['sum', 'count'])
    accuracy_by_level['accuracy'] = accuracy_by_level['sum'] / accuracy_by_level['count']
    overall_accuracy = df['is_correct'].sum() / len(df['is_correct'])
    d=accuracy_by_level['accuracy'].to_dict()
    d['overall_accuracy']=overall_accuracy
    dicts.append(d)
  averages_present = {}
  for key in set().union(*dicts):
      values = [d[key] for d in dicts if key in d]
      averages_present[key] = sum(values) / len(values)
  print(p, averages_present)





op_gsm8k_math_sorted {1: 0.3178294573643411, 2: 0.19259259259259257, 3: 0.1714285714285714, 4: 0.125, 5: 0.029850746268656716, 'overall_accuracy': np.float64(0.138)}
op_math_sorted_ {1: 0.248062015503876, 2: 0.22592592592592595, 3: 0.22539682539682537, 4: 0.10677083333333333, 5: 0.06467661691542288, 'overall_accuracy': np.float64(0.15400000000000003)}
op_gsm8k_math {1: 0.3875968992248062, 2: 0.28888888888888886, 3: 0.20952380952380953, 4: 0.11458333333333333, 5: 0.06965174129353234, 'overall_accuracy': np.float64(0.17733333333333334)}
op_math_ {1: 0.44961240310077527, 2: 0.28148148148148144, 3: 0.1873015873015873, 4: 0.11979166666666667, 5: 0.04477611940298507, 'overall_accuracy': np.float64(0.17133333333333334)}


In [20]:
import pandas as pd

In [30]:
df=pd.read_csv('inf_op_re/op_math_2_final.csv')

In [24]:
df.head()

,original_answer,gold_answer,extracted_answer,extracted_gold,level,subject,is_correct
0,<|begin_of_text|>\n### Instruction:\nBelow is ...,"$\left( 3, \frac{\pi}{2} \right)$","['Interval.open(3, (5*pi)/2)', '( 3, \\frac{5 ...","['(3, pi/2)', '( 3, \\frac{\\pi}{2} )']",2,Precalculus,False
1,<|begin_of_text|>\n### Instruction:\nBelow is ...,$p - q$,"['1', '1']","['p - q', 'p - q']",5,Intermediate Algebra,False
2,<|begin_of_text|>\n### Instruction:\nBelow is ...,$\frac{14}{3}$,"['17/3', '\\frac{17}{3}']","['14/3', '\\frac{14}{3}']",3,Algebra,False
3,<|begin_of_text|>\n### Instruction:\nBelow is ...,$9$,"['8', '8']","['9', '9']",3,Number Theory,False
4,<|begin_of_text|>\n### Instruction:\nBelow is ...,$\text{Evelyn}$,"['angela', '\\text{Angela}']","['evelyn', '\\text{Evelyn}']",2,Algebra,False


In [88]:
df[df['original_answer'].astype(str).str.contains("\nZach", case=False, na=False)]

,original_answer,gold_answer,extracted_answer,extracted_gold,level,subject,is_correct
141,<|begin_of_text|>\n### Instruction:\nBelow is ...,$2$,"['1', '1']","['2', '2']",1,Number Theory,False


In [41]:
df['original_answer'][141]

'<|begin_of_text|>\n### Instruction:\nBelow is a math problem, please solve it as best you can step by step.\n\n### Input:\nZach has three bags and a bunch of pencils to be placed into the bags. He is told to place the greatest number of pencils possible into each of the three bags while also keeping the number of pencils in each bag the same. What is the greatest number of pencils he could have left over?\n\n### Response:\n\nIf Zach has $n$ pencils, then he can place $n/3$ in each of his three bags. Therefore, he has $n = 3n/3 = 3$ left over. This is true for any number of pencils, so the answer is $\\boxed{3}$.\n<|end_of_text|>'

In [44]:
df1['solution'][141]

'If Zach has three or more pencils left over, then he can add another pencil to each bag.  Therefore, Zach can have at most $\\boxed{2}$ pencils left over.'

In [42]:
ls

 combined_run/          gsm8k_sorted/         inf_op/        math_unsorted/
 gsm8k_math_sorted/     gsm8k_unsorted/       inf_op_re/     Progress.gsheet
 gsm8k_math_unsorted/  'Inference Results'/   math_sorted/   stats.pkl


In [43]:
df1= pd.read_csv('Inference Results/combined_5.csv')

In [ ]:
df_sol=

In [27]:
print(df[df['gold_answer']=="$2$"])

                                       original_answer gold_answer  \
45   <|begin_of_text|>\n### Instruction:\nBelow is ...         $2$   
79   <|begin_of_text|>\n### Instruction:\nBelow is ...         $2$   
83   <|begin_of_text|>\n### Instruction:\nBelow is ...         $2$   
88   <|begin_of_text|>\n### Instruction:\nBelow is ...         $2$   
108  <|begin_of_text|>\n### Instruction:\nBelow is ...         $2$   
116  <|begin_of_text|>\n### Instruction:\nBelow is ...         $2$   
141  <|begin_of_text|>\n### Instruction:\nBelow is ...         $2$   
174  <|begin_of_text|>\n### Instruction:\nBelow is ...         $2$   
213  <|begin_of_text|>\n### Instruction:\nBelow is ...         $2$   
215  <|begin_of_text|>\n### Instruction:\nBelow is ...         $2$   
332  <|begin_of_text|>\n### Instruction:\nBelow is ...         $2$   
342  <|begin_of_text|>\n### Instruction:\nBelow is ...         $2$   
363  <|begin_of_text|>\n### Instruction:\nBelow is ...         $2$   
396  <|begin_of_text

In [32]:
df=pd.read_csv('inf_op_re/op_math_1_final.csv')
df['original_answer'][45]

'<|begin_of_text|>\n### Instruction:\nBelow is a math problem, please solve it as best you can step by step.\n\n### Input:\nIn right triangle $ABC$ with $\\angle B = 90^\\circ$, we have $\\sin A = 2\\cos A$.  What is $\\tan A$?\n\n### Response:\n\nThe triangle is shown below:\n\n[asy]\npair A,B,C;\nA = (0,0);\nB = (10,0);\nC = (10,15);\ndraw(A--B--C--A);\ndraw(rightanglemark(C,B,A,23));\nlabel("$A$",A,SW);\nlabel("$B$",B,SE);\nlabel("$C$",C,N);\n[/asy]\n\nWe have $\\sin A = \\frac{BC}{AC}$ and $\\cos A = \\frac{AB}{AC}$, so $\\sin A = 2\\cos A$ gives us $BC = 2AB$.  Then $\\tan A = \\frac{BC}{AB} = \\frac{2AB}{AB} = \\boxed{2}$.\n<|end_of_text|>'

In [40]:
df=pd.read_csv('inf_op_re/op_math_0_final.csv')
df['original_answer'][45]

'<|begin_of_text|>\n### Instruction:\nBelow is a math problem, please solve it as best you can step by step.\n\n### Input:\nIn right triangle $ABC$ with $\\angle B = 90^\\circ$, we have $\\sin A = 2\\cos A$.  What is $\\tan A$?\n\n### Response:\n\nThe triangle is shown below:\n\n[asy]\npair A,B,C;\nA = (0,0);\nB = (10,0);\nC = (10,20);\ndraw(A--B--C--A);\ndraw(rightanglemark(C,B,A,26));\nlabel("$A$",A,SW);\nlabel("$B$",B,SE);\nlabel("$C$",C,N);\n[/asy]\n\nWe have $\\sin A = 2\\cos A$, so $\\frac{AC}{AB} = 2\\cdot \\frac{BC}{AB}$.  Multiplying both sides by $AB$ gives $AC = 2BC$, so $BC = \\frac{AC}{2}$.\n\nThe Pythagorean Theorem gives us $AB^2 = AC^2 + BC^2$.  Substituting $BC = \\frac{AC}{2}$ gives  \\[AB^2 = AC^2 + \\left(\\frac{AC}{2}\\right)^2.\\] This simplifies to  \\[AB^2 = \\frac{5}{4}AC^2,\\] so $\\frac{AB^2}{AC^2} = \\frac{5}{4}$, which means  \\[\\frac{AB}{AC} = \\frac{\\sqrt{5}}{2}.\\] Therefore, $\\tan A = \\frac{AB}{AC} = \\boxed{\\frac{\\sqrt{5}}{2}}$.\n<|end_of_text|>'

In [106]:
ls inf_op_re

op_base_final.csv      op_gsm8k_math_0_final.csv          op_gsm8k_sorted_3_final.csv
op_base_final.json     op_gsm8k_math_0_final.json         op_gsm8k_sorted_3_final.json
op_combined_3.csv      op_gsm8k_math_1_final.csv          op_math_0_final.csv
op_combined_3.json     op_gsm8k_math_1_final.json         op_math_0_final.json
op_combined_4.csv      op_gsm8k_math_2_final.csv          op_math_1_final.csv
op_combined_4.json     op_gsm8k_math_2_final.json         op_math_1_final.json
op_combined_5.csv      op_gsm8k_math_sorted_1_final.csv   op_math_2_final.csv
op_combined_5.json     op_gsm8k_math_sorted_1_final.json  op_math_2_final.json
op_gsm8k_0_final.csv   op_gsm8k_math_sorted_2_final.csv   op_math_sorted_0_final.csv
op_gsm8k_0_final.json  op_gsm8k_math_sorted_2_final.json  op_math_sorted_0_final.json
op_gsm8k_1_final.csv   op_gsm8k_math_sorted_3_final.csv   op_math_sorted_1_final.csv
op_gsm8k_1_final.json  op_gsm8k_math_sorted_3_final.json  op_math_sorted_1_final.json
op_gsm8k_2_fin

In [191]:
df=pd.read_csv('inf_op_re/op_combined_4.csv')


In [139]:
df.head()

,original_answer,gold_answer,extracted_answer,extracted_gold,level,subject,is_correct
0,<|begin_of_text|>\n### Instruction:\nBelow is ...,"$\left( 3, \frac{\pi}{2} \right)$","['0', '0']","['(3, pi/2)', '( 3, \\frac{\\pi}{2} )']",2,Precalculus,False
1,<|begin_of_text|>\n### Instruction:\nBelow is ...,$p - q$,"['1/2', '1/2']","['p - q', 'p - q']",5,Intermediate Algebra,False
2,<|begin_of_text|>\n### Instruction:\nBelow is ...,$\frac{14}{3}$,"['4', '4']","['14/3', '\\frac{14}{3}']",3,Algebra,False
3,<|begin_of_text|>\n### Instruction:\nBelow is ...,$9$,"['7', '7']","['9', '9']",3,Number Theory,False
4,<|begin_of_text|>\n### Instruction:\nBelow is ...,$\text{Evelyn}$,"['0.260000000000000', '0.26']","['evelyn', '\\text{Evelyn}']",2,Algebra,False


'<|begin_of_text|>\n### Instruction:\nBelow is a math problem\n\n### Input:\nA class of 30 students recently took a test.  If 20 students scored 80, 8 students scored 90, and 2 students scored 100, then what was the class average (the mean) on this test?\n\n### Response:\n\nThe total score for the class was $20\\times 80 + 8\\times 90 + 2\\times 100 = 1600 + 720 + 200 = 2620$.  So the class average was $\\frac{2620}{30} = \\boxed{87}$.\n<|end_of_text|>'

In [210]:
import random
i=random.randint(0,499)
# i=227
print(df['extracted_answer'][i],df['extracted_gold'][i])

['134', '134'] ['46', '46']


In [211]:
print(df['original_answer'][i])

<|begin_of_text|>
### Instruction:
Below is a math problem

### Input:
For some value of $x,$ $0 < x < 180,$
\[\tan 53^\circ \tan 81^\circ \tan x^\circ = \tan 53^\circ + \tan 81^\circ + \tan x^\circ.\]Find $x.$

### Response:

From the given equation,
\[\tan 53^\circ \tan 81^\circ \tan x^\circ - \tan 53^\circ - \tan 81^\circ - \tan x^\circ = 0.\]By the tangent addition formula,
\begin{align*}
\tan (53^\circ + 81^\circ - x^\circ) &= \frac{\tan 53^\circ + \tan 81^\circ}{1 - \tan 53^\circ \tan 81^\circ} \\
&= \frac{\tan 53^\circ + \tan 81^\circ}{1 - \tan 53^\circ \tan 81^\circ \tan x^\circ} \\
&= 0.
\end{align*}The only possible value of $53^\circ + 81^\circ - x^\circ$ is a multiple of $90^\circ,$ so $x = \boxed{134}.$
<|end_of_text|>


In [212]:
print(df1['solution'][i])

Isolating $\tan x^\circ,$ we find
\begin{align*}
\tan x &= \frac{\tan 53^\circ + \tan 81^\circ}{\tan 53^\circ \tan 81^\circ - 1} \\
&= -\frac{\tan 53^\circ + \tan 81^\circ}{1 - \tan 53^\circ \tan 81^\circ}.
\end{align*}From the angle addition formula, this is equal to
\[-\tan (53^\circ + 81^\circ) = -\tan 134^\circ = \tan 46^\circ.\]Therefore, $x = \boxed{46}.$


In [170]:
print(df['is_correct'][i])

True


In [214]:
i

68